# Surprise homework

In [1]:
import io 
import os
from collections import defaultdict
import pandas as pd
import numpy as np
from surprise import Dataset
from surprise import SVD
from surprise import get_dataset_dir
from surprise import KNNWithMeans
from surprise import NormalPredictor
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from surprise.model_selection import KFold


### Functions

In [2]:
def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

def precision_recall_at_k(predictions, k=10, threshold=3.5):
    """Return precision and recall at k metrics for each user"""

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    return precisions, recalls

### Load data

In [3]:
data = Dataset.load_builtin('ml-100k')

### Define algorithms

In [4]:
algo_np = NormalPredictor()
algo_KNN_cos = KNNWithMeans(k = 30, sim_options={'name': 'cosine'})
algo_KNN_MSD = KNNWithMeans(k = 30)
algo_KNN_P = KNNWithMeans(k = 30, sim_options={'name': 'pearson'})
algo_SVD = SVD()

### Select algorithm

In [5]:
cv_np = cross_validate(algo_np, data, measures = ['RMSE'], verbose = True)

Evaluating RMSE of algorithm NormalPredictor on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.5230  1.5147  1.5151  1.5231  1.5120  1.5176  0.0046  
Fit time          0.28    0.30    0.24    0.23    0.23    0.25    0.03    
Test time         0.31    0.23    0.27    0.39    0.30    0.30    0.05    


In [6]:
cv_cos = cross_validate(algo_KNN_cos, data, measures = ['RMSE'], verbose = True)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9553  0.9733  0.9542  0.9525  0.9550  0.9581  0.0077  
Fit time          2.77    2.98    2.98    3.07    2.98    2.96    0.10    
Test time         6.43    6.32    6.36    6.08    6.23    6.28    0.12    


In [7]:
cv_msd = cross_validate(algo_KNN_MSD, data, measures = ['RMSE'], verbose = True)


Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9475  0.9456  0.9629  0.9526  0.9551  0.9527  0.0061  
Fit time          0.94    1.03    0.92    0.89    0.98    0.95    0.05    
Test time         7.28    7.39    7.15    6.78    7.66    7.25    0.29    


In [8]:
cv_p = cross_validate(algo_KNN_P, data, measures = ['RMSE'], verbose = True)

Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9505  0.9482  0.9462  0.9535  0.9575  0.9512  0.0040  
Fit time          3.83    4.21    4.26    4.38    4.22    4.18    0.19    
Test time         6.46    7.56    10.01   8.39    6.92    7.87    1.25    


In [9]:
cv_svd = cross_validate(algo_SVD, data, measures = ['RMSE'], verbose = True)

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9425  0.9383  0.9275  0.9338  0.9356  0.9356  0.0050  
Fit time          9.20    10.03   10.47   9.01    9.01    9.54    0.59    
Test time         0.52    0.29    0.24    0.34    0.39    0.36    0.10    


In [10]:
#best - SVD
algo = algo_SVD

### Calculate precision@k and recall@k

In [11]:
trainset, testset = train_test_split(data, test_size=.25)
algo.fit(trainset)
kf = KFold(n_splits=5)
for trainset, testset in kf.split(data):
    algo.fit(trainset)
    predictions = algo.test(testset)
    precisions, recalls = precision_recall_at_k(predictions, k=5, threshold=3.52)

In [12]:
print('precision =', sum(prec for prec in precisions.values()) / len(precisions))
print('recall =', sum(rec for rec in recalls.values()) / len(recalls))

precision = 0.724663835810333
recall = 0.3995996150100657


### Predict

In [41]:
trainset = data.build_full_trainset()
algo.fit(trainset)
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

top_n = get_top_n(predictions, n=5)

In [42]:
user = '31'
res = open("result.txt", "w")
res.write('User ' + user + "\n")
print('User ' + user)
path = get_dataset_dir() + '/ml-100k/ml-100k/u.item'
films_data = pd.read_csv(path, sep="|", encoding='ansi', usecols=[0,1,2], names=['id','name', 'date'])
for id, score in top_n[user]:
    name_date = "({}, {})".format(films_data.at[int(id) - 1, 'name'], films_data.at[int(id) - 1, 'date'])
    print("  {:^5} {:<70} {:^5}".format(id, name_date, round(score, 3)))
    res.write("  {:^5} {:<70} {:^5}".format(id, name_date, round(score, 3))+"\n")
res.close()

User 31
   100  (Fargo (1996), 14-Feb-1997)                                            4.775
   114  (Wallace & Gromit: The Best of Aardman Animation (1996), 05-Apr-1996)  4.746
   98   (Silence of the Lambs, The (1991), 01-Jan-1991)                        4.738
   408  (Close Shave, A (1995), 28-Apr-1996)                                   4.736
   64   (Shawshank Redemption, The (1994), 01-Jan-1994)                        4.716
